In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Dense
from keras_tuner import RandomSearch, Hyperband
from keras.callbacks import ModelCheckpoint

import pandas as pd 

In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("Starter_Code/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df= application_df.drop(["EIN", "NAME"], axis = 1)

In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_counts=application_df["APPLICATION_TYPE"].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
application_types_to_replace = (list(application_counts[application_counts < 500].index))
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [9]:
class_count = application_df["CLASSIFICATION"].value_counts()

In [10]:
class_count[class_count > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [11]:
# Determine which values to replace if counts are less than 1000
classifications_to_replace = list(class_count[class_count < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [12]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
dummified_df = pd.get_dummies(application_df, dtype=int)
dummified_df.head()
# categorical_columns = application_df.select_dtypes(include=["object"]).columns

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [14]:
# Split our preprocessed data into our features and target arrays
X = dummified_df.drop("IS_SUCCESSFUL", axis=1)
y = dummified_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# method to optimize model with hyperparameter options
def create_model(hp):
    model = tf.keras.models.Sequential()
    
    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice("activation", ["relu", "tanh", "sigmoid"])
    
    # Allow kerastuner to decide number of neurons in first layer
    model.add(Dense(units = hp.Int("num_neuron", 1,10, step = 3),
                   activation = activation,
                    input_dim = X_train.shape[1]
                   ))
    
     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for x in range(hp.Int("num_layers", 1,6)):
        model.add(Dense(units = hp.Int("num_layers_" + str(x + 2), 1, 10, step = 2),
                       activation = activation
                       ))
        
    model.add(Dense(units=1, activation = "sigmoid"))
    
    # Compile the model
    model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

    return model

In [17]:
tuner_hyperband = Hyperband(
                    create_model,
                    objective= "val_accuracy",
                    max_epochs= 10,
                    hyperband_iterations=2,
                    overwrite = True
                    )

In [18]:
# Run the kerastuner search for best hyperparameters
tuner_hyperband.search(X_train_scaled, y_train, epochs = 5, validation_data = (X_test_scaled, y_test))

Trial 60 Complete [00h 00m 21s]
val_accuracy: 0.7302623987197876

Best val_accuracy So Far: 0.7332944869995117
Total elapsed time: 00h 12m 37s


In [19]:
# Get best model hyperparameters
best_hyper = tuner_hyperband.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'num_neuron': 10,
 'num_layers': 5,
 'num_layers_2': 5,
 'num_layers_3': 9,
 'num_layers_4': 9,
 'num_layers_5': 5,
 'num_layers_6': 3,
 'num_layers_7': 7,
 'tuner/epochs': 4,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [20]:
# Evaluate best model against full test data
best_model = tuner_hyperband.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f" loss : {round(model_loss,1)*100}%, accuray : {round(model_accuracy,2)*100}%")

268/268 - 1s - loss: 0.5764 - accuracy: 0.7333 - 676ms/epoch - 3ms/step
 loss : 60.0%, accuray : 73.0%


In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model_nn = tf.keras.models.Sequential()

# First hidden layer
model_nn.add(Dense(units = 80, activation = "sigmoid", input_dim = X_train.shape[1]))

# Second hidden layer
model_nn.add(Dense(units = 60, activation="sigmoid"))

# Third hidden layer
model_nn.add(Dense(units = 40, activation="sigmoid"))

# Fourth hidden layer
model_nn.add(Dense(units = 20, activation="sigmoid"))

# Fifth hidden layer
model_nn.add(Dense(units = 5, activation="sigmoid"))
                         
# Output layer
model_nn.add(Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
model_nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3520      
                                                                 
 dense_8 (Dense)             (None, 60)                4860      
                                                                 
 dense_9 (Dense)             (None, 40)                2440      
                                                                 
 dense_10 (Dense)            (None, 20)                820       
                                                                 
 dense_11 (Dense)            (None, 5)                 105       
                                                                 
 dense_12 (Dense)            (None, 1)                 6         
                                                                 
Total params: 11751 (45.90 KB)
Trainable params: 11751

In [22]:
# Compile 
model_nn.compile(loss = "binary_crossentropy", optimizer= "adam", metrics = ["accuracy"])

In [23]:
# Callback to save the Keras model 
callback = ModelCheckpoint(filepath="./checkpoint/checkpoint-{epoch:02d}.keras",
                           save_freq='epoch',
                           verbose=1,
                           save_weights_only = True,
                           period = 5
               )

In [ ]:
# Train
model_nn.fit(X_train_scaled, y_train, epochs = 10, callbacks=[callback])

Epoch 1/10
804/804 [==============================] - 3s 2ms/step - loss: 0.6171 - accuracy: 0.6732
Epoch 2/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5784 - accuracy: 0.7284
Epoch 3/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5759 - accuracy: 0.7288
Epoch 4/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5741 - accuracy: 0.7304
Epoch 5/10
790/804 [============================>.] - ETA: 0s - loss: 0.5743 - accuracy: 0.7295
Epoch 5: saving model to ./checkpoint\checkpoint-05.keras
804/804 [==============================] - 1s 2ms/step - loss: 0.5731 - accuracy: 0.7304
Epoch 6/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5717 - accuracy: 0.7303
Epoch 7/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5650 - accuracy: 0.7296
Epoch 8/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5602 - accuracy: 0.7289
Epoch 9/10
410/804 [==============>...............] - ETA

In [ ]:
# Evaluate
model_loss, model_accuracy = model_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f" loss : {round(model_loss,1)*100}%, accuray : {round(model_accuracy,2)*100}%")

In [ ]:
# Export our model to HDF5 file
model_nn.save("AlphabetSoupCharity.keras")